In [46]:
import pandas as pd
import numpy as np

In [23]:
sales_data_with_wholes=pd.read_excel("/home/jian/Projects/Big_Lots/Newspaper/output_2018-05-25/sales_data_2_year_compariable_0525.xlsx",sheetname=None,dtype=str)
compariable_store_week_sales=sales_data_with_wholes[list(sales_data_with_wholes.keys())[1]]
circ_data=pd.read_csv("/home/jian/Projects/Big_Lots/Newspaper/Analysis/2018-06-06/zip_level_data_revised_2018-06-06.csv",dtype=str)

In [24]:
circ_data=circ_data[['zip_cd','location_id','Store_1','Store_2','2017_compariable_sales','2018_compariable_sales']]

In [25]:
compariable_store_week_sales=compariable_store_week_sales[(compariable_store_week_sales['location_id'].isin(circ_data['Store_1'])) | (compariable_store_week_sales['location_id'].isin(circ_data['Store_2'])) ]




In [26]:
compariable_store_week_sales.shape

(1386, 105)

In [28]:
for col in compariable_store_week_sales.columns.tolist()[1:]:
    compariable_store_week_sales[col]=compariable_store_week_sales[col].astype(float)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
compariable_store_week_sales['Sum']=compariable_store_week_sales[compariable_store_week_sales.columns.tolist()[1:]].sum(axis=1)



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
compariable_store_week_sales.shape

(1386, 106)

In [61]:
del compariable_store_week_sales['Sum']

In [62]:
compariable_store_week_sales.head(2)

,location_id,2016-05-14,2016-05-21,2016-05-28,2016-06-04,2016-06-11,2016-06-18,2016-06-25,2016-07-02,2016-07-09,...,2018-03-03,2018-03-10,2018-03-17,2018-03-24,2018-03-31,2018-04-07,2018-04-14,2018-04-21,2018-04-28,2018-05-05
0,1,69777.95,71434.38,84109.90,97543.29,74738.07,73443.62,79688.32,91539.51,86258.46,...,110956.73,100013.22,99194.00,91914.46,78716.28,99790.03,112517.34,71202.97,83309.33,92893.51
1,3,55028.57,54031.78,58181.12,68837.29,49062.13,56143.93,45296.41,67724.16,65967.84,...,107881.07,98716.61,73944.82,69992.44,67088.99,79300.64,76779.62,57830.55,60531.27,64324.36


In [82]:
count_df=pd.DataFrame()
for location_id,group in compariable_store_week_sales.groupby("location_id"):
    df=group.copy().reset_index()
    del df['index']
    df['Count']=np.nan
    k=0
    for col in compariable_store_week_sales.columns.tolist()[1:]:
       if df[col][0]>0:
        k=k+1
    df['Count'][0]=k/2
    count_df=count_df.append(df)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [83]:
circ_data_count=circ_data[~pd.isnull(circ_data['Store_1'])]
count_df=count_df[['location_id','Count']]

In [84]:
count_df=count_df.rename(columns={"location_id":"Store_1","Count":"Count_1"})
count_df_Store_1_Count=pd.merge(circ_data_count,count_df,on="Store_1",how="left")

In [85]:
count_df=count_df.rename(columns={"Store_1":"Store_2","Count_1":"Count_2"})
count_df_Store_1and2_Count=pd.merge(count_df_Store_1_Count,count_df,on="Store_2",how="left")

In [86]:
count_df_Store_1and2_Count.head(2)

,zip_cd,location_id,Store_1,Store_2,2017_compariable_sales,2018_compariable_sales,Count_1,Count_2
0,01060,[1493],1493,NaN,3341402.3,3428432.25,52.0,NaN
1,01089,[1863],1863,NaN,4653944.98,4403450.290000001,52.0,NaN


In [87]:
count_df_Store_1and2_Count['Store_Count']=np.where(pd.isnull(count_df_Store_1and2_Count['Store_2']),1,2)

In [88]:
count_df_Store_1and2_Count.head(2)

,zip_cd,location_id,Store_1,Store_2,2017_compariable_sales,2018_compariable_sales,Count_1,Count_2,Store_Count
0,01060,[1493],1493,NaN,3341402.3,3428432.25,52.0,NaN,1
1,01089,[1863],1863,NaN,4653944.98,4403450.290000001,52.0,NaN,1


In [90]:
count_df_Store_1and2_Count['Avg_Week_Count']=count_df_Store_1and2_Count[['Count_1','Count_2']].sum(axis=1)/count_df_Store_1and2_Count['Store_Count']

In [92]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Newspaper/Analysis/Avg_Week_Count_For_Store_20180610.xlsx",engine="xlsxwriter")
count_df_Store_1and2_Count.to_excel(writer,"Count",index=False)
compariable_store_week_sales.to_excel(writer,"Sales_Data",index=False)
writer.save()



